# 📝 Einführung zur Code-Qualität: Demonstration von 5 Ruff-Regelgruppen

## 💡 Was ist Ruff?

Ruff ist ein extrem schneller Python-Linter und Formatter, der **Best Practices** beim Schreiben von Code durchsetzt und häufige Fehler, Ineffizienzen oder Sicherheitslücken identifiziert. Dieses Notebook demonstriert die Anwendung von fünf verschiedenen Regelgruppen von Ruff (**S, E, D, PERF**) anhand konkreter Beispiele.

## Die demonstrierten Regelgruppen und Themen:

1. **Security (S101, S506)**: Fokus auf die Vermeidung von Sicherheitsrisiken und unzuverlässigem Verhalten.
2. **Formatierung/Style (E711)**: Einhaltung des PEP 8 Style Guides für bessere Lesbarkeit.
3. **Dokumentation (D100)**: Sicherstellen einer adäquaten Code-Dokumentation.
4. **Performance (PERF203)**: Identifizierung von Code-Stellen mit unnötigem Performance-Overhead.

Für jede Regelgruppe werden folgende Schritte durchgeführt:
- Erzeugung eines Beispiels mit **fehlerhaftem Code**
- Erstellung einer **korrigierten Variante**
- **Prüfung mit Ruff** und Darstellung der Ergebnisse

Die Ausführung kann sowohl in **Google Colab** als auch in einem lokalen **Jupyter Notebook** erfolgen.

## 🛠️ Vorbereitung: Installation von Ruff

In [1]:
# Installation von ruff (nur ausführen, wenn nicht installiert)
!pip install ruff --quiet

## Regelgruppe Security (S)
---
### Fehler: Unsichere Nutzung von `assert` (*S101*)

In [2]:
# Fehlerhafte Version (unsicher)
# Ruff erkennt S101, da 'assert' in optimiertem Modus (-O) ignoriert wird und
# somit kritische Validierungen fehlschlagen können.
with open("s101_assert_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Unsichere Nutzung von assert (kann in optimiertem Modus ignoriert werden)
user_input = ""
assert user_input != "", "Input darf nicht leer sein!"  # unsicher
""")

# Ruff-Check (Security-Regeln, Auswahl S)
!ruff check --select S s101_assert_bad.py

print("--------------------------------------------")

# Korrigierte Version (sicher)
# Stattdessen wird eine explizite 'if'-Bedingung mit 'raise' verwendet,
# was immer garantiert ist.
with open("s101_assert_good.py", "w", encoding="utf-8") as f:
    f.write("""# Sichere Nutzung mit if + Exception
user_input = ""
if user_input == "":
    raise ValueError("Input darf nicht leer sein!")  # sicher
""")

# Ruff-Check (Security-Regeln)
!ruff check --select S s101_assert_good.py


S101 Use of `assert` detected
 --> s101_assert_bad.py:3:1
  |
1 | # Unsichere Nutzung von assert (kann in optimiertem Modus ignoriert werden)
2 | user_input = ""
3 | assert user_input != "", "Input darf nicht leer sein!"  # unsicher
  | ^^^^^^
  |

Found 1 error.
--------------------------------------------
All checks passed!


---
### Fehler: Unsicheres `yaml.load()` (*S506*)

In [3]:
# Fehlerhafte Version (unsicher)
# S506 wird ausgelöst, da 'yaml.load' die Instanziierung beliebiger Python-Objekte ermöglicht
# und somit ein Sicherheitsrisiko (Remote Code Execution) darstellt.
with open("s506_yaml_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Unsichere Nutzung von yaml.load (führt zu Sicherheitsrisiken)
import yaml

data = "!!python/object/apply:os.system ['echo unsicher']"
result = yaml.load(data)  # unsicher
print(result)
""")

# Ruff-Check (Security-Regeln)
!ruff check --select S s506_yaml_bad.py

print("--------------------------------------------")

# Korrigierte Version (sicher)
# Die sichere Alternative 'yaml.safe_load' wird verwendet.
with open("s506_yaml_good.py", "w", encoding="utf-8") as f:
    f.write("""# Sichere Nutzung von yaml.safe_load
import yaml

data = "a: 1\\nb: 2"
result = yaml.safe_load(data)
print(result)
""")

# Ruff-Check (Security-Regeln)
!ruff check --select S s506_yaml_good.py


S506 Probable use of unsafe `yaml.load`. Allows instantiation of arbitrary objects. Consider `yaml.safe_load`.
 --> s506_yaml_bad.py:5:10
  |
4 | data = "!!python/object/apply:os.system ['echo unsicher']"
5 | result = yaml.load(data)  # unsicher
  |          ^^^^^^^^^
6 | print(result)
  |

Found 1 error.
--------------------------------------------
All checks passed!


## Regelgruppe Formatierung / Style (E)
---
### Fehler: Falscher Vergleich mit `None` (*E711*)

In [4]:
# Fehlerhafte Version (falscher Vergleich mit None)
# E711 wird ausgelöst, da PEP 8 vorschreibt, 'None' immer mit 'is' oder 'is not' zu vergleichen.
with open("e711_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Falscher Vergleich mit None
x = None
if x == None:
    print("x ist None")
""")

# Ruff-Check (nur Formatierungs-/Style-Regeln, Auswahl E)
!ruff check --select E e711_bad.py

print("--------------------------------------------")

# Korrigierte Version (richtiger Vergleich mit None)
with open("e711_good.py", "w", encoding="utf-8") as f:
    f.write("""# Richtiger Vergleich mit None
x = None
if x is None:
    print("x ist None")
""")

# Ruff-Check (nur Formatierungs-/Style-Regeln)
!ruff check --select E e711_good.py


E711 Comparison to `None` should be `cond is None`
 --> e711_bad.py:3:9
  |
1 | # Falscher Vergleich mit None
2 | x = None
3 | if x == None:
  |         ^^^^
4 |     print("x ist None")
  |
help: Replace with `cond is None`

Found 1 error.
No fixes available (1 hidden fix can be enabled with the `--unsafe-fixes` option).
--------------------------------------------
All checks passed!


## Regelgruppe Dokumentation (D)
---
### Fehler: Fehlender Modul-Docstring (*D100*)

In [5]:
# Fehlerhafte Version (kein Modul-Docstring)
# D100 wird ausgelöst, da die Datei (das Modul) keinen Docstring in der ersten Zeile enthält.
with open("d100_bad.py", "w", encoding="utf-8") as f:
    f.write("""def greet(name):
    \"\"\"Gibt einen Begrüßungstext aus.\"\"\"
    print(f"Hallo {name}")
""")

# Ruff-Check nur für D100
!ruff check --select D100 d100_bad.py

print("--------------------------------------------")

# Korrigierte Version (mit Modul-Docstring)
with open("d100_good.py", "w", encoding="utf-8") as f:
    f.write("""\"\"\"Dieses Modul enthält eine Begrüßungsfunktion.\"\"\"

def greet(name):
    \"\"\"Gibt einen Begrüßungstext aus.\"\"\"
    print(f"Hallo {name}")
""")

# Ruff-Check nur für D100
!ruff check --select D100 d100_good.py


D100 Missing docstring in public module
--> d100_bad.py:1:1

Found 1 error.
--------------------------------------------
All checks passed!


## Regelgruppe Performance (PERF)
---
### Fehler: `try/except` in einer Schleife (*PERF203*)

In [6]:
# Fehlerhafte Version (try/except in einer Schleife → ineffizient)
# PERF203 wird ausgelöst, da das Exception Handling in jedem Schleifendurchlauf
# unnötigen Overhead verursacht.
with open("perf203_bad.py", "w", encoding="utf-8") as f:
    f.write("""# Schlechte Performance: try/except in der Schleife
for i in range(5):
    try:
        print(int("x"))  # Fehler provozieren
    except ValueError:
        print("Fehler bei Umwandlung")
""")

# Ruff-Check für Performance-Regeln
!ruff check --select PERF perf203_bad.py

print("--------------------------------------------")

# Korrigierte Version (try/except um die Schleife → effizienter)
with open("perf203_good.py", "w", encoding="utf-8") as f:
    f.write("""# Bessere Performance: try/except außerhalb der Schleife
try:
    for i in range(5):
        print(int("x"))  # Fehler provozieren
except ValueError:
    print("Fehler bei Umwandlung")
""")

# Ruff-Check für Performance-Regeln
!ruff check --select PERF perf203_good.py


PERF203 `try`-`except` within a loop incurs performance overhead
 --> perf203_bad.py:5:5
  |
3 |       try:
4 |           print(int("x"))  # Fehler provozieren
5 | /     except ValueError:
6 | |         print("Fehler bei Umwandlung")
  | |______________________________________^
  |

Found 1 error.
--------------------------------------------
All checks passed!


## 📈 Interpretation und Zusammenfassung
---
Die vorangegangenen Beispiele demonstrieren, dass **Ruff** ein unverzichtbares Werkzeug für die moderne Python-Entwicklung ist, da es weit über die grundlegende Syntaxprüfung hinausgeht.

### Wichtigste Erkenntnisse:

* **Sicherheit (S-Regeln)**: Die Regeln **S101** (`assert` in produktivem Code) und **S506** (`yaml.load`) helfen, kritische Sicherheitslücken zu vermeiden, die zu unzuverlässigem Verhalten oder im schlimmsten Fall zur **Remote Code Execution** führen können.
* **Standardisierung und Lesbarkeit (E- & D-Regeln)**: Die Einhaltung von Konventionen wie **E711** (`is None`) und **D100** (Modul-Docstrings) verbessert die Lesbarkeit und **Wartbarkeit** des Codes erheblich, was essenziell für Teamarbeit ist.
* **Effizienz (PERF-Regeln)**: Die Regel **PERF203** (Exception Handling in Schleifen) zeigt, dass Ruff auch auf **Performance-Engpässe** hinweist. Die Verlagerung des `try/except`-Blocks um die gesamte Schleife reduziert den Overhead des Exception Handlings.

Zusammenfassend unterstützt Ruff Entwickler aktiv dabei, Code zu schreiben, der **sicher, performant und konventionskonform** ist, und sorgt so für eine höhere Qualität in jedem Python-Projekt.